# NY Taxi Fare Prediction

## Final Competition - Group 34


In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import xgboost as xgb
from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from typing import Callable

import os
# input_dir = '/kaggle/input/'
# working_dir = '/kaggle/working/'
input_dir = r'.\input'
working_dir = r'.\working'
WEATHER = True
TAXICAB = False


In [ ]:
# Reading Training Data
N_ROWS = 30_000_000
df_train = pd.read_csv(os.path.join(input_dir, 'new-york-city-taxi-fare-prediction/train.csv'),
                       nrows=N_ROWS, parse_dates=["pickup_datetime"])
if TAXICAB:
    df_taxi = pd.read_csv(os.path.join(input_dir, 'taxicab_dis_train_85000.csv'),
                        nrows=N_ROWS)
    df_train = pd.concat([df_train, df_taxi], axis= 1)

df_train_copy = df_train.copy()
df_train.head()


## Preprocessing


### Data Cleansing (For Training Data)


In [ ]:
# remove any data with out-of-bound pickup or dropoff location (coordinate outside NYC)
def remove_out_of_bound(df_input: pd.DataFrame,
                        longitude_bounds: list = [-75, -72],
                        latitude_bounds: list = [40, 42]) -> pd.DataFrame:
    pickup_in_bound = ((df_input.pickup_longitude > longitude_bounds[0]) &
                       (df_input.pickup_longitude < longitude_bounds[1]) &
                       (df_input.pickup_latitude > latitude_bounds[0]) &
                       (df_input.pickup_latitude < latitude_bounds[1]))
    dropoff_in_bound = ((df_input.dropoff_longitude > longitude_bounds[0]) &
                        (df_input.dropoff_longitude < longitude_bounds[1]) &
                        (df_input.dropoff_latitude > latitude_bounds[0]) &
                        (df_input.dropoff_latitude < latitude_bounds[1]))
    return df_input[pickup_in_bound & dropoff_in_bound]

# remove any data negative fare


def remove_negative_fare(df_input: pd.DataFrame) -> pd.DataFrame:
    return df_input.drop(df_input[df_input['fare_amount'] < 0].index, axis=0)

# remove any data with unrealistic passenger count


def limit_max_passenger(df_input: pd.DataFrame, max_passenger: int = 6) -> pd.DataFrame:
    return df_input.drop(df_input[df_input['passenger_count'] > max_passenger].index, axis=0)

# preprocessing pipeline containing all cleansing steps


def remove_all_invalid_data(df_input: pd.DataFrame) -> pd.DataFrame:
    print('Before Cleansing: {}'.format(len(df_input)))
    df_input = df_input.dropna(how='any', axis='rows')
    print('After Removing NaN values: {}'.format(len(df_input)))
    df_input = remove_out_of_bound(df_input)
    print('After Removing Out-of-Bounds: {}'.format(len(df_input)))
    df_input = remove_negative_fare(df_input)
    print('After Removing Negative Fares: {}'.format(len(df_input)))
    df_input = limit_max_passenger(df_input)
    print('After Limiting Max Passengers: {}'.format(len(df_input)))
    return df_input


## Feature Engineering


### Distance Features


In [ ]:
# linear distance between pickup and dropoff location
def euclidean_dist(pickup_lat: float, pickup_long: float,
                   dropoff_lat: float, dropoff_long: float) -> float:
    LAT2KM = 110.574
    LONG2KM = 111.320
    distance = np.sqrt(((dropoff_lat - pickup_lat) * LAT2KM) ** 2 +
                       ((dropoff_long - pickup_long) * LONG2KM) ** 2)
    return distance

# shortest distance between pickup and dropoff location on a sphere


def haversine_dist(pickup_lat: float, pickup_long: float,
                   dropoff_lat: float, dropoff_long: float) -> float:
    dLat = (dropoff_lat - pickup_lat) * np.pi / 180.0
    dLon = (dropoff_long - pickup_long) * np.pi / 180.0
    lat1 = (pickup_lat) * np.pi / 180.0
    lat2 = (dropoff_lat) * np.pi / 180.0

    a = (np.power(np.sin(dLat / 2), 2) +
         np.power(np.sin(dLon / 2), 2) *
         np.cos(lat1) * np.cos(lat2))
    RAD = 6371
    distance = 2 * np.arcsin(np.sqrt(a))
    return RAD * distance


def get_distance(df_input: pd.DataFrame,
                 distance_func: Callable[[float, float, float, float], float]) -> pd.DataFrame:
    return distance_func(df_input['pickup_latitude'], df_input['pickup_longitude'], df_input['dropoff_latitude'], df_input['dropoff_longitude'])

    '''
    return df_input.apply(lambda row :distance_func(row['pickup_latitude'],
                                                     row['pickup_longitude'],
                                                     row['dropoff_latitude'],
                                                     row['dropoff_longitude'],), axis = 1)
    '''


### Time and Date Features


In [ ]:
# split datetime datatype to separate columns
def get_date_time(df_input: pd.DataFrame, time_col_name: str = 'pickup_datetime') -> pd.DataFrame:
    df_datetime = pd.DataFrame()
    df_datetime['year'] = df_input[time_col_name].dt.year
    df_datetime['month'] = df_input[time_col_name].dt.month
    df_datetime['date'] = df_input[time_col_name].dt.day
    df_datetime['day_of_week'] = df_input[time_col_name].dt.dayofweek
    df_datetime['hour'] = df_input[time_col_name].dt.hour
    return df_datetime

# encode cyclic features into sine and consine components


def cyclical_encoding(df_input: pd.DataFrame, cyclic_features: list = ['month', 'date', 'day_of_week', 'hour']) -> pd.DataFrame:
    max_val_dict = {'month': 12, 'date': 31, 'day_of_week': 7, 'hour': 23}
    df_cyclic = pd.DataFrame()
    for feature in cyclic_features:
        df_cyclic[feature +
                  '_sin'] = np.sin(2 * np.pi * df_input[feature]/max_val_dict[feature])
        df_cyclic[feature +
                  '_cos'] = np.cos(2 * np.pi * df_input[feature]/max_val_dict[feature])
    return df_cyclic


def is_after_price_hike(df_input: pd.DataFrame) -> pd.DataFrame:
    PRICE_HIKE_DAY = dt.datetime(2012, 10, 1, 0, 0, 0, 0, dt.timezone.utc)
    return df_input.pickup_datetime > PRICE_HIKE_DAY


def is_rush_hour(df_input: pd.DataFrame) -> pd.DataFrame:
    rush_hour = (((df_input['hour'] > 15) & (df_input['hour'] < 20)) & (
        (df_input['day_of_week'] > 0) & (df_input['day_of_week'] < 6)))
    return rush_hour


def is_overnight(df_input: pd.DataFrame) -> pd.DataFrame:
    overnight = (((df_input['hour'] > 21) | (df_input['day_of_week'] < 6)))
    return overnight


### Location-Based Features


In [ ]:
def get_dist_to_airports(df_input: pd.DataFrame) -> pd.DataFrame:
    jfk_coord = (40.639722, -73.778889)
    ewr_coord = (40.6925, -74.168611)
    lga_coord = (40.77725, -73.872611)

    pickup_lat = df_input['pickup_latitude']
    dropoff_lat = df_input['dropoff_latitude']
    pickup_lon = df_input['pickup_longitude']
    dropoff_lon = df_input['dropoff_longitude']

    pickup_jfk = haversine_dist(
        pickup_lat, pickup_lon, jfk_coord[0], jfk_coord[1])
    dropoff_jfk = haversine_dist(
        jfk_coord[0], jfk_coord[1], dropoff_lat, dropoff_lon)
    pickup_ewr = haversine_dist(
        pickup_lat, pickup_lon, ewr_coord[0], ewr_coord[1])
    dropoff_ewr = haversine_dist(
        ewr_coord[0], ewr_coord[1], dropoff_lat, dropoff_lon)
    pickup_lga = haversine_dist(
        pickup_lat, pickup_lon, lga_coord[0], lga_coord[1])
    dropoff_lga = haversine_dist(
        lga_coord[0], lga_coord[1], dropoff_lat, dropoff_lon)

    df_distances = pd.DataFrame()

    df_distances['jfk_dist'] = pd.concat(
        [pickup_jfk, dropoff_jfk], axis=1).min(axis=1)
    df_distances['ewr_dist'] = pd.concat(
        [pickup_ewr, dropoff_ewr], axis=1).min(axis=1)
    df_distances['lga_dist'] = pd.concat(
        [pickup_lga, dropoff_lga], axis=1).min(axis=1)

    return df_distances


### Utilities


In [ ]:
def remove_duplicate_columns(df_input: pd.DataFrame) -> pd.DataFrame:
    return df_input.loc[:, ~df_input.columns.duplicated()]


def add_all_features(df_input: pd.DataFrame) -> pd.DataFrame:
    # distance features
    df_input['euclidean_distance'] = get_distance(df_input, euclidean_dist)
    df_input['haversine_distance'] = get_distance(df_input, haversine_dist)

    # time features
    df_input = remove_duplicate_columns(
        pd.concat([df_input, get_date_time(df_input)], axis=1))
    df_input = remove_duplicate_columns(
        pd.concat([df_input, cyclical_encoding(df_input)], axis=1))
    
    # time and regulation features
    df_input['is_after_price_hike'] = is_after_price_hike(df_input)
    df_input['is_rush_hour'] = is_rush_hour(df_input)
    df_input['is_overnight'] = is_overnight(df_input)
    
    # airport vicinity features
    df_input = remove_duplicate_columns(
        pd.concat([df_input, get_dist_to_airports(df_input)], axis=1))
    
    return df_input

def standard_scaling(df_input: pd.DataFrame, scaler= StandardScaler()) -> pd.DataFrame:
    df_scaled_input = scaler.transform(df_input)
    return pd.DataFrame(df_scaled_input, columns = df_input.columns)   

## Main Code


In [ ]:
# for quick start
df_train = df_train_copy.copy()

In [ ]:
if (WEATHER == True):
    # get weather features
    df_weather = pd.read_csv(os.path.join(input_dir, 'open-meteo/weather.csv'),
                            parse_dates=["time"])
    df_weather = remove_duplicate_columns(
        pd.concat([df_weather, get_date_time(df_weather, time_col_name= 'time')], axis=1))
    df_weather.head()


In [ ]:
# Preprocessing
df_train = remove_all_invalid_data(df_train)


In [ ]:
# Feature Engineering
# distance features
df_train = add_all_features(df_train)

if WEATHER == True:
    df_train =  pd.merge(df_train, df_weather, on=['year', 'month', 'date', 'day_of_week', 'hour'])
    df_train['weathercode'] = df_train['weathercode'].astype('category')

df_train.columns

In [ ]:
# Fixing Broken Taxicab Distance
if TAXICAB:
    taxi_NA_idx = (df_train['taxicab_distance']==-1) | (df_train['taxicab_distance']==0)
    df_train.loc[taxi_NA_idx, 'taxicab_distance'] = df_train['euclidean_distance'][taxi_NA_idx]*1000

In [ ]:
# Training
df_train_keys = df_train['key']
df_train_Y = df_train['fare_amount']

unnecessary_cols = ['key', 'pickup_datetime', 'fare_amount']
if WEATHER == True:
    unnecessary_cols.append('time')
df_train_X = df_train.drop(unnecessary_cols, axis=1)


In [ ]:
scaler = StandardScaler()
scaler.fit(df_train_X)
df_train_X = standard_scaling(df_train_X, scaler)

    'learning_rate': [0.01, 0.05, 0.12, 0.3],
    'max_depth': [6, 7, 8],
    'min_child_weight': [1, 5, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],


In [ ]:
# hyperparameters optimization
FIND_HYPERPARAMETER = False
if FIND_HYPERPARAMETER:
    xgb_model = xgb.XGBRegressor()
    parameters = {
        # tunable parameters
        'learning_rate': [0.05],
        'max_depth': [5, 8, 10],
        'min_child_weight': [1, 5, 15, 200],
        'subsample': np.arange(0.5, 1.0, 0.1),
        'colsample_bytree': np.arange(0.4, 1.0, 0.1),
        'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
        'n_estimators': [100, 500, 1000],


        # untunable parameters
        'objective': ['reg:squarederror'],
        'nthread': [8],
        'predictor': ['gpu_predictor'],
        'verbosity': [0],
        'seed': [1]
    }

    clf = RandomizedSearchCV(xgb_model, param_distributions=parameters, n_jobs=1, n_iter=25,
                             scoring='neg_root_mean_squared_error', verbose=5, refit=True)

    clf.fit(df_train_X, df_train_Y)
    cv_best_param = clf.best_params_
    xgb_model_best = clf.best_estimator_
    print(cv_best_param)

''' 
best params
{'colsample_bytree': 0.6,
 'colsample_bylevel': 0.7,
 'learning_rate': 0.05,
 'max_depth': 8,
 'min_child_weight': 1,
 'n_estimators': 1000,
 'nthread': 8,
 'objective': 'reg:squarederror',
 'predictor': 'gpu_predictor',
 'seed': 1,
 'subsample': 1.0,
 'verbosity': 0}
'''
'''
{'verbosity': 0, 'subsample': 0.8999999999999999, 'seed': 1, 'predictor': 'gpu_predictor', 'objective': 'reg:squarederror', 'nthread': 8, 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 10, 'learning_rate': 0.05, 'colsample_bytree': 0.6, 'colsample_bylevel': 0.7}
'''

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_train_X, df_train_Y, random_state=56, test_size=0.1)

In [ ]:
# training
best_param = {'colsample_bytree': 0.8,
              'colsample_bylevel': 0.8,
              'learning_rate': 0.01,
              'max_depth': 8,
              'min_child_weight': 1,
              'n_estimators': 1000,
              'subsample': 0.9,
              
              'nthread': 8,
              'objective': 'reg:squarederror',
              'predictor': 'gpu_predictor',
              'seed': 1,
              'subsample': 0.8,
              'booster': 'gbtree',
              'verbosity': 0}

dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical= True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical= True)

n = 10000
model = xgb.train(
    params=best_param,
    dtrain=dtrain_reg,
    num_boost_round=n,
    early_stopping_rounds=50,

    evals=[(dtrain_reg, "train"), (dtest_reg, "validation")]
)

preds = model.predict(dtest_reg)
rmse = mean_squared_error(y_test, preds, squared=False)
print(rmse)


In [ ]:
FIND_CAT_HYPERPARAMETER = False

if FIND_CAT_HYPERPARAMETER:
        grid = {'depth': [6, 10, 14],
                'l2_leaf_reg': [0.5, 5, 15, 30]}

        cat_model = CatBoostRegressor(random_state=3,
                                l2_leaf_reg=30,
                                iterations=10000,
                                early_stopping_rounds=50,
                                eval_metric='RMSE',
                                # use_best_model=True,
                                task_type="GPU")
        cat_model.grid_search(grid, X_train, y_train,
                        search_by_train_test_split=True, plot=True)

In [ ]:
CATBOOST = False
if CATBOOST:
        grid = {'depth': [6, 10, 14],
                'l2_leaf_reg': [0.5, 5, 15, 30]}

        cat_model = CatBoostRegressor(random_state=3,
                                depth=10,
                                l2_leaf_reg=30,
                                iterations=10000,
                                early_stopping_rounds=50,
                                eval_metric='RMSE',
                                use_best_model=True,
                                task_type="GPU")

        cat_model.fit(X_train,y_train, eval_set=(X_test, y_test), verbose=1, plot=True)

In [ ]:
CONTINUE = False
# continue training (optional)
if CONTINUE:
    n = 10000
    model = xgb.train(
        params=best_param,
        dtrain=dtrain_reg,
        num_boost_round=n,
        early_stopping_rounds=100,
        xgb_model=model,
        evals=[(dtrain_reg, "train"), (dtest_reg, "validation")]
    )

    preds = model.predict(dtest_reg)
    rmse = mean_squared_error(y_test, preds, squared=False)
    print(rmse)



In [ ]:
df_test = pd.read_csv(os.path.join(
    input_dir, 'new-york-city-taxi-fare-prediction/test.csv'), parse_dates=["pickup_datetime"])
if TAXICAB:
    df_taxi = pd.read_csv(os.path.join(input_dir, 'taxicab_dis_test.csv'),)
    df_test = pd.concat([df_test, df_taxi], axis= 1)
df_test.head()


In [ ]:
df_test = add_all_features(df_test)
if WEATHER == True:
    df_test =  pd.merge(df_test, df_weather, on=['year', 'month', 'date', 'day_of_week', 'hour'])
    df_test['weathercode'] = df_test['weathercode'].astype('category')


df_test_keys = df_test['key']
unnecessary_cols = ['key', 'pickup_datetime']
if WEATHER == True:
    unnecessary_cols.append('time')
df_test_X = df_test.drop(unnecessary_cols, axis=1)

df_test_X = standard_scaling(df_test_X, scaler)

In [ ]:
df_test_X

In [ ]:
df_test_Y = model.predict(xgb.DMatrix(df_test_X))
df_test_submit = pd.DataFrame()
df_test_submit['key'] = df_test_keys
df_test_submit['fare_amount'] = df_test_Y
df_test_submit


In [ ]:
if CATBOOST:
    df_test_Y = cat_model.predict(df_test_X)
    df_test_submit = pd.DataFrame()
    df_test_submit['key'] = df_test_keys
    df_test_submit['fare_amount'] = df_test_Y
    df_test_submit

In [ ]:
df_test_submit.to_csv(os.path.join(
    working_dir, '010_submission.csv'), index=False)
model.save_model(os.path.join(working_dir, '010.model'))


In [ ]:
import json
configs = json.loads(model.save_config())
configs['learner']

In [ ]:
from matplotlib import pyplot

feature_importance = model.get_score(importance_type='gain')
# feature importance
print(feature_importance)

# plot
keys = list(feature_importance.keys())
values = list(feature_importance.values())

data = pd.DataFrame(data=values, index=keys, columns=[
                    "score"]).sort_values(by="score", ascending=False)
data.nlargest(50, columns="score").plot(kind='barh', figsize=(20, 10))
